# Neural Network Prediction of Sahelian Summer Rainfall
***

#### Resources:
* [Mardata Course](https://github.com/mardatade/Course-Python-for-Machine-Learning/blob/master/3.%20Neural%20Network.ipynb)
* [Keras for Data Scientists](https://keras.io/getting_started/intro_to_keras_for_engineers/#data-loading-amp-preprocessing)

In [1]:
import numpy as np
import pandas as pd 
import xarray as xr

import matplotlib.pyplot as plt
%matplotlib inline

import scipy.stats as st

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn import metrics


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import tensorflow_addons as tfa

from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

import dask
from dask import delayed

<br>
<br>

## 1. Data Loading & Preprocessing
***

<br>

### a) Loading & Normalization

**predictor:** contains the data used for the inputs  
**label:** from Sahelrainfall data serves as validation data

In [2]:
predictor = xr.open_dataset('data/da_pred_all.nc').to_dataframe()

predictor_unit = pd.DataFrame(
    data = StandardScaler().fit_transform(predictor), 
    columns = predictor.columns,
    index =  predictor.index
)


# load validatoin data (Summer Rainfall over Sahel and scale to [cm/month]) 
labels = np.mean(np.loadtxt("data/da_o_sahelprecip19012017.txt", skiprows=8,)[:,7:10] * 0.01,  axis=1)

predictor_unit.head()

,siod_e,siod_w,sst_med,tsa,tna,sst_mdr,sata_lnh,sata_lsh,sata_onh,sata_osh,slp_darwin,slp_tahiti,amo,nao,pdo,np,nino12,nino3,nino34,nino4
year,,,,,,,,,,,,,,,,,,,,
1901,-1.100027,-1.152764,-0.745530,-0.595366,0.388372,0.608415,-0.123443,-0.732091,-0.497808,-0.737797,0.074807,1.634819,0.923204,0.917456,-0.193321,1.938388,-0.950168,-0.595561,-0.214314,-0.079270
1902,0.088643,0.340415,-1.507314,-0.954566,-0.346586,-0.173588,-1.289978,-0.201810,-1.175314,-0.987096,1.443896,2.682485,-0.620146,-1.172590,0.819716,-0.162154,0.991321,0.969845,1.099218,1.070532
1903,-0.900789,0.669332,-2.243639,-2.186294,-0.101970,0.283583,-1.333183,-1.076056,-1.415719,-1.333946,-0.071881,1.535042,-0.458290,-1.030410,-0.186187,0.530864,-0.371251,0.000784,0.524139,0.842095
1904,-0.949568,-1.056219,-0.079925,-1.975498,-2.214111,-1.894743,-1.135674,-1.133384,-1.863746,-1.778347,-0.903114,1.235708,-1.872482,1.447076,-0.892459,0.756497,-0.307712,-0.234313,-0.475713,-0.741738
1905,-0.034350,-0.632249,-0.718895,-1.684676,-1.334312,-1.014906,-1.314666,-0.595938,-1.284589,-0.954579,0.759351,-2.655622,-0.499163,-1.289888,0.545055,-0.326007,1.227830,1.497381,1.439037,1.032459


<br>

### b) PCA

In [3]:
# Scikit PCA transformation
pca = PCA()
principalComponents = pca.fit_transform(predictor_unit)


# Create Create Pandas DF from PCs
col = []
for i in range(1, 21):
    col.append(f'PC{i}')

predictor_pc = pd.DataFrame(
    data = principalComponents,
    columns = col,
    index =  predictor.index
)

# Test for unit-variance and zero mean:
# np.std(pred_pc)
# np.mean(pred_pc)
# pred_pc.head()

predictor_pc.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20
year,,,,,,,,,,,,,,,,,,,,
1901,-1.568434,-0.814406,-1.565711,-1.237071,1.472947,-0.593357,-1.660153,0.222967,0.241259,0.837099,0.098283,-0.767361,-0.099698,0.875102,-0.391004,0.067938,-0.012741,0.176348,0.188827,0.012466
1902,-0.521670,2.782115,-0.365347,2.152565,2.364542,0.248512,-1.855028,-0.231364,0.177858,0.788062,-0.135515,0.526785,-0.439029,0.236945,0.613192,-0.173206,-0.049516,0.003676,0.020811,-0.022700
1903,-2.357779,1.783520,-1.194862,1.368700,1.273662,-1.280782,-1.766921,0.441110,-0.412690,0.913848,-1.644854,0.649893,-0.240188,0.273618,0.151983,0.152255,-0.063746,0.036216,0.128500,0.007985
1904,-5.168501,0.642722,1.054827,-0.759804,1.351015,1.020594,-0.413493,-0.695127,-0.606953,-0.289135,-0.644362,0.201273,-0.556953,0.270713,0.003502,-0.383464,0.148037,-0.071637,0.241812,0.073306
1905,-1.282539,4.379097,0.640607,0.433165,-0.940971,-1.433710,1.068731,-1.191238,0.581742,-0.393126,-0.344745,-0.274467,0.226232,-0.207231,0.295327,-0.322379,0.324228,-0.052399,0.166036,0.116238


<br>
<br>

## 2. MODEL SETUP AND TUNING
***

<br>

### Clear Logs
***

In [4]:
rm -rf logs/*

<br>

### Hyperparameter Selection
***

In [18]:
###################################
#####EXAMPLE SETUP FOR TESTING#####
###################################


#GRID SERACH HYPERPARAMETER#
#---------------------------
HP_INPUT_VAR_NINE = hp.HParam('input_var_nine', hp.Discrete(['PC9']),display_name='9th Input Variable')
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['AdamW']),display_name='Optimizer')
HP_LEARN_RATE = hp.HParam('learn_rate', hp.Discrete([0.1]),display_name='Learning Rate')
HP_WEIGHT_DECAY = hp.HParam('weight_decay', hp.Discrete([1e-1]),display_name='Weight Decay')
HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1]),display_name='Batch Size')
HP_EPOCHS = hp.HParam('n_epochs', hp.Discrete([10, 20]),display_name='Epochs')


#CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
#----------------------------------------------------
cv_param={
    'N_FOLDS': 2,         # number of folds -> small for Test Runs
    'TEST_FRAC': .1    # factrion that is held out for test
}


#BAGGING PARAMETER (NO PART OF GRID SEARCH)#
#-------------------------------------------
n_baggs = 5  # number of baggs -> small for test runs





####################
#####FULL SETUP#####
####################


# #GRID SERACH HYPERPARAMETER#
# #---------------------------
# HP_INPUT_VAR_NINE = hp.HParam('input_var_nine', hp.Discrete(['PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16']),display_name='9th Input Variable')
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['AdamW', 'SGDW']),display_name='Optimizer')
# HP_LEARN_RATE = hp.HParam('learn_rate', hp.Discrete([0.01, 0.1, 0.2]),display_name='Learning Rate')
# HP_WEIGHT_DECAY = hp.HParam('weight_decay', hp.Discrete([0.001, 0.01, 0.1]),display_name='Weight Decay')
# HP_BATCH_SIZE = hp.HParam('batch_size', hp.Discrete([1, 3, 10, 30]),display_name='Batch Size')
# HP_EPOCHS = hp.HParam('n_epochs', hp.Discrete([30, 80, 120]),display_name='Epochs')


# #CROSS VALIDATION PARAMETER (NO PART OF GRID SEARCH)#
# #----------------------------------------------------
# cv_param={
#     'N_FOLDS': 105,      # number of folds -> sample size as in Badr
#     'TEST_FRAC': .1    # factrion that is held out for test
# }


# #BAGGING PARAMETER (NO PART OF GRID SEARCH)#
# #-------------------------------------------
# n_baggs = 10 # number of baggs -> 10 as in Badr

<br>

### Metric Selection
---

In [19]:
METRIC_TRAIN_MSE_MU= 'train_mse_mu'
METRIC_TRAIN_MSE_SIG= 'train_,mse_sig'
METRIC_TRAIN_CORR_MU= 'train_corr_mu'
METRIC_TRAIN_CORR_SIG= 'train_corr_sig'

METRIC_TEST_MSE_MU= 'test_mse_mu'
METRIC_TEST_MSE_SIG= 'test_mse_sig'
METRIC_TEST_CORR_MU= 'test_corr_mu'
METRIC_TEST_CORR_SIG= 'test_corr_sig'

<br>

### Log Experiment Confiuration to TensorBoard
---

In [20]:
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_INPUT_VAR_NINE, HP_OPTIMIZER, HP_LEARN_RATE, HP_WEIGHT_DECAY, HP_BATCH_SIZE, HP_EPOCHS],
        metrics=[
            hp.Metric(METRIC_TRAIN_MSE_MU, display_name='Training Sample MSE µ'),
#             hp.Metric(METRIC_TRAIN_MSE_SIG, display_name='Training Sample  MSE σ'),
#             hp.Metric(METRIC_TRAIN_CORR_MU, display_name='Training Sample Correlation µ'),
#             hp.Metric(METRIC_TRAIN_CORR_SIG, display_name='Training Sample  Correlation σ'),
            hp.Metric(METRIC_TEST_MSE_MU, display_name='Test Sample MSE µ'),
#             hp.Metric(METRIC_TEST_MSE_SIG, display_name='Test Sample  MSE σ'),
#             hp.Metric(METRIC_TEST_CORR_MU, display_name='Test Sample Correlation µ'),
#             hp.Metric(METRIC_TEST_CORR_SIG, display_name='Test Sample  Correlation σ')
        ],
    )

<br>

### Build Model Function
---

In [21]:
def BuildModel(hparams):      
    
    
    model = keras.Sequential([
            layers.Dense(3, activation="sigmoid", name="layer1", input_shape=(9,)),
            layers.Dense(1, activation='linear', name='output')
        ])
    
    model.compile(
        loss='mean_squared_error',
        optimizer=getattr(tfa.optimizers, hparams[HP_OPTIMIZER])(
            learning_rate=hparams[HP_LEARN_RATE],
            weight_decay=hparams[HP_WEIGHT_DECAY]
        )
    )
    return model

<br>

### Bagging Function
---

In [22]:
def Bagging(hparams, features, model, train_index, test_index):
    
    
    
    # set emty output matrices
    y_train_bagging = np.zeros((train_index.size, n_baggs))
    y_test_bagging = np.zeros((test_index.size, n_baggs))    
    
    
    #Train the model 'n_baggs' times and store model predictions into matrice
    for n in range(n_baggs):
        
#         print ('baggin run', n)
#         print ('PREDICTION ON TEST DATA:', y_test_bagging)
        
        # Bootstrap sampling from training Data with Size(Training Data)
        train_index_bootstrap = np.random.choice(train_index, train_index.size)

        #Train the model 
        model.fit(
            features[train_index_bootstrap],
            labels[train_index_bootstrap],
            batch_size=hparams[HP_BATCH_SIZE],
            epochs=hparams[HP_EPOCHS],
            verbose=0
        )
        
        #Run the model for insample data and store in one matrix:
        y_train_bagging[:, n] = np.squeeze(model.predict(features[train_index]))
        
        # ... and for out of sample data        
        y_test_bagging[:, n] = np.squeeze(model.predict(features[test_index]))

    # return mean of the outputs over baggins (1st dimension)
    return y_train_bagging.mean(1), y_test_bagging.mean(1)
#     return y_test_bagging.mean(1)

<br>

### Cross Validation Training & Error Calculation Funktion
---

In [23]:
def TrainModel(hparams, cv_param, predictor_pc, labels):
        
    
#     train_mse = np.empty(cv_param['N_FOLDS'])
#     train_corr = np.empty(cv_param['N_FOLDS'])
    
#     test_mse = np.empty(cv_param['N_FOLDS'])
#     test_corr = np.empty(cv_param['N_FOLDS'])
    train_mse = []
    test_mse = []
    
    
    #choose Inputs
    features = predictor_pc.loc[:,['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', hparams[HP_INPUT_VAR_NINE]]].to_numpy()
    
    
    
    
    #Cross Validation#
    ##################
    
    cv_fold = 0
    
    for train_index, test_index in ShuffleSplit(n_splits=cv_param['N_FOLDS'], test_size=cv_param['TEST_FRAC']).split(features):
        
#         print(cv_fold)
#         print("TRAIN:", train_index, "TEST:", test_index)
        
    
        # Build the model according to definition:
        model = BuildModel(hparams)
        
        # Train and predict using Bagging    
        y_train_test = delayed(Bagging)(hparams, features, model, train_index, test_index)
        
        
        #Compute error metrics for in sample data
        train_err = y_train_test[0] - labels[train_index]
        train_mse.append(np.mean(train_err**2))
        
        # ... and for out of sample data
        test_err =  y_train_test[1] - labels[test_index]
        test_mse.append(np.mean(test_err**2))
        
        
#         print ( "BAGGING OUT Test", y_test)
        cv_fold += 1
    
    #######################################################################################################
    result_test = delayed(np.mean)(test_mse)
    result_train= delayed(np.mean)(train_mse)
    
    train_mse_mu, test_mse_mu = dask.compute(result_test, result_train)
    #Error Moments#
    ###############
    
    eval_metrics = {
        'train_mse_mu': train_mse_mu,
#         'train_mse_sig': np.std(train_mse),
#         'train_corr_mu': np.mean(train_corr),
#         'train_corr_sig': np.std(train_corr),            
        'test_mse_mu': test_mse_mu,
#         'test_mse_sig': np.std(test_mse),
#         'test_corr_mu': np.mean(test_corr),
#         'test_corr_sig': np.std(test_corr),
    }
    
#     print(eval_metrics)
    
    return eval_metrics

<br>

### Model Run and Log Function
---

In [24]:
def RunModel(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        
        eval_metrics = TrainModel(hparams, cv_param, predictor_pc, labels)
        tf.summary.scalar(METRIC_TRAIN_MSE_MU,   eval_metrics['train_mse_mu'],   step=1)
#         tf.summary.scalar(METRIC_TRAIN_MSE_SIG,  eval_metrics['train_mse_sig'],  step=1)
#         tf.summary.scalar(METRIC_TRAIN_CORR_MU,  eval_metrics['train_corr_mu'],  step=1)
#         tf.summary.scalar(METRIC_TRAIN_CORR_SIG, eval_metrics['train_corr_sig'], step=1)
        tf.summary.scalar(METRIC_TEST_MSE_MU,    eval_metrics['test_mse_mu'],    step=1)
#         tf.summary.scalar(METRIC_TEST_MSE_SIG,   eval_metrics['test_mse_sig'],   step=1)
#         tf.summary.scalar(METRIC_TEST_CORR_MU,   eval_metrics['test_corr_mu'],   step=1)
#         tf.summary.scalar(METRIC_TEST_CORR_SIG,  eval_metrics['test_corr_sig'],  step=1)

<br>

### Grid Search
---

In [25]:
%%time

session_num = 0        

for input_var_nine in HP_INPUT_VAR_NINE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
        for learn_rate in HP_LEARN_RATE.domain.values:
            for weight_decay in HP_WEIGHT_DECAY.domain.values:
                for batch_size in HP_BATCH_SIZE.domain.values:
                    for n_epochs in HP_EPOCHS.domain.values:


                        hparams = {
                            HP_INPUT_VAR_NINE: input_var_nine,
                            HP_OPTIMIZER: optimizer,
                            HP_LEARN_RATE: learn_rate,
                            HP_WEIGHT_DECAY: weight_decay,
                            HP_BATCH_SIZE: batch_size,
                            HP_EPOCHS: n_epochs,                
                        }

                        run_name = f"run-{session_num}"
                        print(f'--- Starting trial: {run_name}')
                        print({h.name: hparams[h] for h in hparams})
                        RunModel('logs/hparam_tuning/' + run_name, hparams)
                        session_num += 1    

--- Starting trial: run-0
{'input_var_nine': 'PC9', 'optimizer': 'AdamW', 'learn_rate': 0.1, 'weight_decay': 0.1, 'batch_size': 1, 'n_epochs': 10}
--- Starting trial: run-1
{'input_var_nine': 'PC9', 'optimizer': 'AdamW', 'learn_rate': 0.1, 'weight_decay': 0.1, 'batch_size': 1, 'n_epochs': 20}
CPU times: user 23.2 s, sys: 2.91 s, total: 26.1 s
Wall time: 10.1 s


<br>
<br>

***
***
<br>
<br>